In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Урок 9. Языковое моделирование

#### Задание
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [3]:
import tensorflow as tf

import numpy as np
import os
import time

In [4]:
text = open('extract_dialogues_from_anekdots.txt', 'rb').read().decode(encoding='utf-8')
print(text[:200])

- В окно в Европу поставили крепкую железную решётку...
- Мы не закончили, это - арматура!



- Нет ли здесь среди вас дзен-буддистов из Омска?
- Есть, земляк, я!
- А, может, стукнемся где-нибудь в це


In [5]:
print('Length of text: {} characters'.format(len(text)))

Length of text: 3097801 characters


In [6]:
text = text[:int((len(text)/2))]

Сделаем предсказание текста на основе посимвольного предсказания

In [7]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

176 unique characters


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

-
 
В
 
о


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
BATCH_SIZE = 512

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(512, 200, 176) # (batch_size, sequence_length, vocab_size)


In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (512, 200, 176)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.17026


In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
history = model.fit(dataset, epochs=50)

Epoch 1/50
15/15 [==============================] - 52s 3s/step - loss: 4.7913
Epoch 2/50
15/15 [==============================] - 48s 3s/step - loss: 3.3532
Epoch 3/50
15/15 [==============================] - 49s 3s/step - loss: 2.9701
Epoch 4/50
15/15 [==============================] - 48s 3s/step - loss: 2.7623
Epoch 5/50
15/15 [==============================] - 49s 3s/step - loss: 2.6273
Epoch 6/50
15/15 [==============================] - 49s 3s/step - loss: 2.4722
Epoch 7/50
15/15 [==============================] - 48s 3s/step - loss: 2.3674
Epoch 8/50
15/15 [==============================] - 49s 3s/step - loss: 2.3013
Epoch 9/50
15/15 [==============================] - 48s 3s/step - loss: 2.2357
Epoch 10/50
15/15 [==============================] - 49s 3s/step - loss: 2.1710
Epoch 11/50
15/15 [==============================] - 48s 3s/step - loss: 2.1102
Epoch 12/50
15/15 [==============================] - 49s 3s/step - loss: 2.0555
Epoch 13/50
15/15 [==============================

In [21]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 30

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [22]:
text_ = generate_text(model, start_string=u"Сказал один мужик другому ")
print(text_)

Сказал один мужик другому побе и эка вала зо вача, о Не 


Сделаем предсказание текста на основе пословное предсказания

In [23]:
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re



In [24]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

text = preprocess_text(text)

In [25]:
vocab = sorted(set(text.split()))
print('{} unique characters'.format(len(vocab)))

20216 unique characters


In [26]:
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in text.split()])

In [27]:
text_as_int, text, len(text_as_int), len(text)

(array([10718,  5045, 13155, ...,  9322,  7284,  1334]),
 'окно европа поставить крепкий железный решётка мын закончить арматура среди дзенбуддист омск земляк стукнуться гденибыть центр пивко попить дхарма попередавать корейский язык эквивалент русский идти х судить отвечать америка должный откуда номер вчера аня дать поздравлять откуда номер спать красавица че страшный выспаться фуу нога вонять дак жпы расти тон вонять рука понюхать психлечебница мочь человек 6 утро будить игра труба желательно госпитализировать адрес воинский часть казарма дневальный компания самсунг слушать вчера купить сверхмощный пылесос плохо слышный говорить громкий аркаша взять замуж люся дать поспать свадьба вчера отгулять выродок ублюдок сволочиненавидеть ааааа дмитрий анатолиевич точно член совет безопасность кончить ага высокий означать жёлтый одинаковый палка логотип параллельный импорт наклониться обходить санкция слева владимир владимирович дело… случиться 9 июнь германия снег выпасть твой мать мой клима

In [28]:
seq_length = 10
examples_per_epoch = len(text)//(seq_length+1)


word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [29]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [30]:
dataset = sequences.map(split_input_target)

In [31]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [32]:
vocab_size = len(vocab)


embedding_dim = 256


rnn_units = 1024

In [33]:
model_2 = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model_2.compile(optimizer='adam', loss=loss)

In [34]:
history = model_2.fit(dataset, epochs=50)

Epoch 1/50
21/21 [==============================] - 11s 341ms/step - loss: 9.3647
Epoch 2/50
21/21 [==============================] - 8s 342ms/step - loss: 8.7230
Epoch 3/50
21/21 [==============================] - 8s 347ms/step - loss: 8.6012
Epoch 4/50
21/21 [==============================] - 8s 357ms/step - loss: 8.5759
Epoch 5/50
21/21 [==============================] - 8s 361ms/step - loss: 8.5639
Epoch 6/50
21/21 [==============================] - 8s 354ms/step - loss: 8.5499
Epoch 7/50
21/21 [==============================] - 8s 349ms/step - loss: 8.5326
Epoch 8/50
21/21 [==============================] - 8s 349ms/step - loss: 8.5121
Epoch 9/50
21/21 [==============================] - 8s 345ms/step - loss: 8.4910
Epoch 10/50
21/21 [==============================] - 8s 347ms/step - loss: 8.4763
Epoch 11/50
21/21 [==============================] - 8s 351ms/step - loss: 8.4616
Epoch 12/50
21/21 [==============================] - 8s 351ms/step - loss: 8.4482
Epoch 13/50
21/21 [=====

In [35]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 5

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string.split() if s in word2idx]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [39]:
text_ = generate_text(model_2, start_string=u"Cказал один мужик другому ")
print(text_)

Cказал один мужик другому рубль ухо спасть следующий дать


При обучении посимвольно генируется только без связный набор знаков. При токонизировании слов, предложение начинает появлятся смысол в предложении.